
# EECS 469/569: Homework 2
## OpenMP
## Single Node Performance of Roaring Thunder
### Due: Sunday, Oct. 17 *before* midnight

### Instructions

This is a partner assignment (you should work together on all sections); if there are an odd number of students there can be exactly one group of three. A ***formal*** report is **not** required for this homework. The results should be included where noted in this Jupyter notebook by the keyword **DELIVERABLE**. All supporting files required to run the Jupyter notebook must be included in your submission. All figures should be clear, with axes labeled, including a legend, and a caption. All figures should have a short description and referenced in text in the context of the assignment. This assignment will be graded tougher than HW01, make sure you answer **WHY** things happened, not just **WHAT**. 

Copy this Jupyter notebook (rename it to hw02_yourlastnames) to its own folder in your user space on the Roaring Thunder cluster and only include files relevant to this assignment in the folder. There is information in Lecture Slides 6 and 7 on how to run the Jupyter notebook on the cluster. 

**(-5 points if not on time) DELIVERABLE:** Email Dr. Hansen with your partner for the assignment before midnight on Monday, Oct. 11 (CC your partner, only send one email per group).  DONE

### Objectives
1. familiarize the student with OpenMP (Open Multiprocessing);
2. analyze overhead required for OpenMP; and
3. apply knowledge of OpenMP to speedup the linear algebra code from Homework 1.

 **(5 points) <span style="color:red;">FINAL DELIVERABLE:**</span> ***After*** you have completed the entire assignment, write a few paragraphs on your main takeaways from the assignment. **Clearly state** how the work was split up between you and your partner. 

# Procedure

# 1. (Optional) Introduction to OpenMP

**This section is optional,** there are no deliverables or points associated with this section. If you are having issues with OpenMP, this may be of use to you to get a refresher. 

1. Go to [HPC Training Moodle](https://www.hpc-training.org/xsede/moodle/), create an account, and take the Introduction to OpenMP course. There are many other intro courses that may assist you as we continue with the class.

# 2. OpenMP Overhead

The process of forking/joining threads is not free! There is an overhead (in terms of time/CPU cycles) involved in each OpenMP directive and clause. This section will explore the overhead of these different OpenMP constructs using the [Edinburgh Parallel Computing Centre (EPCC) OpenMP MicroBenchmark Suite](https://www.epcc.ed.ac.uk/research/computing/performance-characterisation-and-benchmarking).

1. Read the paper "Measuring Synchronisation and Scheduling Overheads in OpenMP" on D2L. 
2. Make sure that you have copied the `syncbench` folder. 
3. Compile and run the `syncbench` microbenchmark using the provided makefile and SLURM file (you may need to do this in a separate command line window, from the syncbench folder). 
```bash
make syncbench
sbatch hw2_sync.slurm
```
4. Once the batch file has completed, read through the output of the benchmark.

**(5 points) <span style="color:red;">DELIVERABLE:**</span> Write a one paragraph summary of the paper. 



**(15 points) <span style="color:red">DELIVERABLE:**</span> Create a table with the average overhead time, and discuss the overhead of the following OpenMP directives: parallel, for, parallel for, barrier, critical, atomic. 

|Directive   |Overhead Time Ms|
|:-----------|:---------------|
|Parallel    | 12.788068|
|For         | 6.595064 |
|Parallel For| 12.128117|
|Barrier     | 6.681943 |
|Critical    | 0.551266 |
|Atomic      | 0.098452 | 

The sections with parallel took the longest overhead time beacause the threads have to be spawned at that point. With critcal and atomic the parallel section is already defined and there is very little overhead required. while the barrier is included in a parallel section each thread has to execute the statement. The for has a smaller overhead than the parallel for because similar to the critcal and atomic the threads ar already spwaned. The for does have overhead becasue it has to divide and assign the work among the threads.

# 3. OpenMP Linear Algebra

## ⚠️⚠️ **WARNING:** All deliverables must be executed on a node obtained through SLURM for this section ⚠️⚠️

In this section, we will begin to explore the speedup gained through OpenMP parallelism using your linear algebra code from Homework 1. Start by making copies of the working versions of your ***WORKING*** Homework 1 code (if you had errors, you need to fix it):
* Matrix-Matrix Multiply Transpose: mat_mat_omp.c
* Matrix-Vector Product: mat_vec_omp.c
* Dot Product: dot_omp.c

## 3.1 OpenMP Matrix-Matrix Product
1. Add an `omp parallel for` section to your outermost matrix multiplication loop. 
    * Recall that only the outermost loop variable is made private
    * Ensure thread safety of your result matrix $\mathbf{C}$
    * While testing, you may use the head node of the cluster as long as you use a small matrix size $N$ and number of threads $T$
    * To ensure your code is correct, compare the output to a known $i,j$ from Homework 1
2. Using a dedicated SLURM node (`--nodes=1`, `--ntasks-per-node=40`), run and time the matrix-matrix product using a matrix size of $N=4096$ and different numbers of threads $T = {2,4,8,16,32,40}$. 
    * HINT: you may want to write a Bash for-loop in your SLURM file that sets the number of threads and then runs the program. 
    * HINT: you may want to add an additional column in the output .csv file to specify the number of threads

In [15]:
%env OMP_NUM_THREADS=10
!echo $OMP_NUM_THREADS

env: OMP_NUM_THREADS=10
10


In [30]:
!gcc matrix_multiply_transpose_serial.c -o matmult_s -lm -fopenmp
!./matmult_s 1024 non-parallel.dat

Performed a 1024 x 1024 matrix multiply transpose in 4.338874 seconds
Number of floating point operations = 2 * 1024^3 = 2147483648
Flops = 4.949404e+08
Element from Matrix C[4][5]: 74924.664062


In [90]:
!gcc matrix_multiply_transpose_parallel.c -o matmult_p -lm -fopenmp
!./matmult_p 1024 parallel.dat

Performed a 1024 x 1024 matrix multiply transpose in 0.603840 seconds
Number of floating point operations = 2 * 1024^3 = 2147483648
Flops = 3.556381e+09
Element from Matrix C[4][5]: 74924.664062


<span style="color:red;">**(30 points) DELIVERABLE:**</span> Link to your final '.c' code here: []()

Create three figures that have $T$ on the x-axis, and on the y-axis:
* parallel speedup (Use your $T=1$ time from Homework 1 from the serial version of the transpose code. If you had an error in your transpose, you should correct it and get a new time.)
* floating point operations per second (FLOPs)
* execution time

***USE AN APPROPRIATE SI PREFIX FOR YOUR Y-AXES!*** Discuss in one paragraph per figure the impact of OpenMP and the number of threads on algorithm performance. ***WHY*** do you think you are seeing the results you are?

### 3.1.1 Optimized Matrix-Matrix Product

Using any OpenMP or other methods, for $T=40$ and $N=4096$, obtain the fastest possible time for your matrix-matrix product (while still getting the correct solution). 

**(5 points, based on final speed) <span style="color:red">DELIVERABLE:**</span> Describe how you obtained your fastest result. Link to your final '.c' code here: []()



In [89]:
!gcc matrix_multiply_transpose_speed.c -o matmult_f -O2 -lm -fopenmp
!./matmult_f 4096 parallel.dat

Performed a 4096 x 4096 matrix multiply transpose in 8.791791 seconds
Number of floating point operations = 2 * 4096^3 = 137438953472
Flops = 1.563265e+10
Element from Matrix C[4][5]: 258400.421875


## 3.2 Comparison of Different Linear Algebra Algorithms

In the prior section we explored matrix-matrix multiply, which is an $N^3$ algorithm on $N^2$ data. In this section, you will explore the scalability of the dot product ($N$ operations on $N$ data) and the matrix-vector product ($N^2$ operations on $N^2$ data). 

1. Speedup your matrix-vector product and dot product using OpenMP. Ensure you get the same answer as the serial version. 
2. Using a dedicated SLURM node (`--nodes=1`, `--ntasks-per-node=40`), design an experiment to evaluate the scalability of the two algorithms with respect to number of threads.

In [51]:
!gcc ./matrix_vector_multiply.c -o matvec -fopenmp -lm
!./matvec 4096

Performed a 4096 x 4096 matrix vector multiply in 0.045802 seconds
Number of floating point operations = 2 * 4096^3 = 137438953472
Flops = 3.000713e+12
value 4 from Matc: 279472.218750

In [77]:
!gcc ./matrix_vector_multiply_speed.c -o matvec_s -fopenmp -lm -O3
!./matvec_s 4096

Performed a 4096 x 4096 matrix vector multiply in 0.003569 seconds
Number of floating point operations = 2 * 4096^3 = 137438953472
Flops = 3.851115e+13
value 4 from Matc: 279472.218750

**(20 points) <span style="color:red">DELIVERABLE:** </span>Design an experiment to evaluate the scalability of the two algorithms with respect to number of threads. Compare the two algorithms with the matrix-matrix product. Include any discussion and figures required to support your conclusions. Include a link to your commented source code here: []()

# 4. Other OpenMP Stuff

## 4.1 More OpenMP

Read the latest [OpenMP 5.0 "cheat sheet"](https://www.openmp.org/wp-content/uploads/OpenMPRef-5.0-111802-web.pdf). Find a directive or clause that we did not use in class or this assignment, research what it does, and implement a small test case to show its operation. **Do not just copy an example from online.** 

**(10 points) <span style="color:red">DELIVERABLE</span>:** Link your '.c' code here: []()
1. Write a paragraph on the new OpenMP directive/clause that you used, what it does, and how you used it in your code.
2. Discuss with another group that chose a different OpenMP directive/clause than you. Write a few sentences about it and which group (by name of team members). 

## 4.2 Solve a Cool Problem with OpenMP

Solve any problem that interests you using OpenMP. 

**(10 points) <span style="color:red">DELIVERABLE:**</span> Describe what problem you chose and how you accelerated it using OpenMP. Discuss your problem with a different group than **Deliverable 4.1** and describe their problem in a few sentences, and which group (by name of team members). Link your '.c' code here: []()



# SUBMISSION INSTRUCTIONS

Zip your .ipynb, all source files (you do not need to submit syncbench), and any necessary data files as 'HW02_XX.zip' (or .7z), where 'XX' is your HW02 group number. Email the .zip file to Dr. Hansen, one per group, and CC your partner. You need to ensure that all figures properly show up in your .ipynb that you email to Dr. Hansen. This needs to be in Dr. Hansen's inbox before midnight on Sunday, Oct. 17. 

Submit a .pdf (e.g., print via Chrome as .pdf) to the D2L dropbox before the due date as well.